# Development of functions to: 1. Find retro steps for input smiles target. 2. Exctract info needed to populate DB models 3. Extract robotic action steps and populate DB

In [1]:
from rdkit import Chem
from rxn4chemistry import RXN4ChemistryWrapper
import os
import time

In [2]:
# Setup IBM RxN API
api_key=os.environ['IBM_API_KEY'] 
rxn4chemistry_wrapper = RXN4ChemistryWrapper(api_key=api_key)
rxn4chemistry_wrapper.create_project('Test actions')

{'response': {'payload': {'id': '5fbbd36ea965f9000134eb86',
   'metadata': {},
   'embed': {},
   'computedFields': {},
   'createdOn': 1606144878436,
   'createdBy': '5421668d-2db4-4064-a1e6-f77e33938145',
   'modifiedOn': 1606144878436,
   'modifiedBy': '5421668d-2db4-4064-a1e6-f77e33938145',
   'name': 'Test actions',
   'description': None,
   'attempts': [],
   'visibility': None},
  'metadata': {'uiMessages': {'errors': [], 'infos': [], 'warnings': []},
   'extendedPagination': {}}}}

In [3]:
import pandas as pd

In [38]:
test =pd.read_csv('/home/warren/XChem_projects/car-django-react/Test_DB_files/csv_upload.csv')
test

,Targets,Hello
0,Cc1ccccc1-c1cccc(NC(=O)Cc2cccc(O)c2)c1\t,NaN
1,COc1ccc(C)cc1NC(=O)Cc1cccc(O)c1\t,NaN
2,Oc1ccccc1NC(=O)Cc1cccc(O)c1\t,NaN
3,O=C(Cc1cccc(O)c1)Nc1ccc(F)cc1F\t,NaN
4,Cc1cc(C)c(NC(=O)Cc2cccc(O)c2)c(C)c1\t,NaN
5,O=C(Cc1cccc(O)c1)NCC1COc2ccccc2O1\t,NaN
6,O=C(Cc1cccc(O)c1)Nc1ccccc1OC(F)(F)F\t,NaN
7,Cn1cc(NC(=O)Cc2cccc(O)c2)cn1\t,NaN
8,O=C(Cc1cccc(O)c1)Nc1cn[nH]c1\t,NaN
9,CC(NC(=O)Cc1cccc(O)c1)c1ccccc1Cl\t,NaN


In [78]:
#from celery import shared_task 

import pandas as pd

# CSV check functions
def add_warning(target_name, field, warning_string, validate_dict):
    validate_dict['target_name'].append(target_name)
    validate_dict['field'].append(field)
    validate_dict['warning_string'].append(warning_string)

    return validate_dict

def checkColumnName(column_name, validate_dict):
    if column_name != 'Targets':
        validate_dict = add_warning(target_name = 'Column name error',
                                    field = 'column_name',
                                    warning_string = "Column name set to {} and should be 'Targets'".format(column_name),
                                    validate_dict=validate_dict)
    return validate_dict


def checkNumberColumns(columns, validate_dict):
    no_columns = len(columns)
    
    if no_columns > 1:
        validate_dict = add_warning(target_name = 'Column name error',
                                    field = 'column_name',
                                    warning_string = "Found {} column names. Set and name columns to 'Targets' only".format(no_columns),
                                    validate_dict=validate_dict)
        
    if no_columns == 1:
        validate_dict = checkColumnName(columns[0], validate_dict)
    
    return validate_dict


def checkSMILES(target_smiles, index, validate_dict):
    
    mol = Chem.MolFromSmiles(target_smiles)
    
    if mol is None:
        validate_dict = add_warning(target_name = target_smiles,
                            field = 'smiles_check',
                            warning_string = "Input target smiles: '{}' at index {} is not a valid smiles".format(target_smiles, index),
                            validate_dict=validate_dict)

    return validate_dict


# @shared_task
def validateFileUpload(csv_fp, project_details):
    """ Celery task to process validate the uploaded files for retrosynthesis planning.
    
    Parameters
    ----------
    csv_fp: str
        filepath of the uploaded csv file, which is saved to temporary storage by `viewer.views.UploadCSV`
    project_details: dict
        dictionary of project details (name, email and project_name) that will be used to create the project model
        if the csv file is validated
    
    Returns
    -------
    validate_output: tuple
        contains the following:
            - validate dict (dict): dict containing any errors found during the validation step
            - validated (bool): True if the file(s) were validated, False if not
            - filename (str): name of the uploaded csv file
    """
    
    validated = True
    
    validate_dict = {'target_name': [],
                     'field': [],
                     'warning_string': []}
    
    # Open csv file as Pandas df
    uploaded_csv_df = pd.read_csv(csv_fp)
    
    # Check no of column headings and name of column heading
    columns = uploaded_csv_df.columns
    validate_dict = checkNumberColumns(columns, validate_dict)
    
    if len(validate_dict['target_name']) != 0:
        validated = False
    else:
        # Check SMILES
        indexes = [i for i,smi in enumerate(uploaded_csv_df['Targets'])]
        smiles_list = [smi for i, smi in enumerate(uploaded_csv_df['Targets'])]
        
        for smi, index in zip(smiles_list, indexes): 
            validate_dict = checkSMILES(smi, index, validate_dict)

    if len(validate_dict['target_name']) !=0:
        validated = False
        
    return (validate_dict, validated, csv_fp, smiles_list, project_details)



In [79]:
validateFileUpload('/home/warren/XChem_projects/car-django-react/Test_DB_files/csv_upload.csv')

RDKit ERROR: [18:23:48] SMILES Parse Error: syntax error while parsing: Targets
RDKit ERROR: [18:23:48] SMILES Parse Error: Failed parsing SMILES 'Targets' for input: 'Targets'


({'target_name': ['Targets'],
  'field': ['smiles_check'],
  'warning_string': ["Input target smiles: 'Targets' at index 13 is not a valid smiles"]},
 False,
 '/home/warren/XChem_projects/car-django-react/Test_DB_files/csv_upload.csv',
 ['Cc1ccccc1-c1cccc(NC(=O)Cc2cccc(O)c2)c1\t',
  'COc1ccc(C)cc1NC(=O)Cc1cccc(O)c1\t',
  'Oc1ccccc1NC(=O)Cc1cccc(O)c1\t',
  'O=C(Cc1cccc(O)c1)Nc1ccc(F)cc1F\t',
  'Cc1cc(C)c(NC(=O)Cc2cccc(O)c2)c(C)c1\t',
  'O=C(Cc1cccc(O)c1)NCC1COc2ccccc2O1\t',
  'O=C(Cc1cccc(O)c1)Nc1ccccc1OC(F)(F)F\t',
  'Cn1cc(NC(=O)Cc2cccc(O)c2)cn1\t',
  'O=C(Cc1cccc(O)c1)Nc1cn[nH]c1\t',
  'CC(NC(=O)Cc1cccc(O)c1)c1ccccc1Cl\t',
  'O=C(Cc1cccc(O)c1)Nc1cccnc1\t',
  'COC(=O)c1sccc1NC(=O)Cc1cccc(O)c1\t',
  'COc1ccc2nc(NC(=O)Cc3cccc(O)c3)sc2c1\t',
  'Targets'])

In [ ]:
def getRetroSyn(smiles):
        # NB need to add method_id to function call via views and urls -
        # see - https://stackoverflow.com/questions/59842406/how-to-pass-an-id-to-a-view-as-an-argument-in-django
        """
        Use the IBM API to get some possible retrosynthesis routes
        """
        # Create dummy dictionary to create while loop to catch when status is a SUCCESS
        results = {}
        results['status'] = None
        reactions = None

        while reactions is None:  
            try:
                time.sleep(30)
                response = rxn4chemistry_wrapper.predict_automatic_retrosynthesis(product=self.smiles)
                while results['status'] != 'SUCCESS': 
                    time.sleep(30)
                    results = rxn4chemistry_wrapper.get_predict_automatic_retrosynthesis_results(response['prediction_id'])
                    reactions = results
            except Exception as e:
                print(e)
        return reactions
    
def createReactionModel(smiles):
    # Function that takes in all the info and creates a reaction object

    # Create Reaction object
    reaction = Reaction()
    # Need the method_id which is passed in as obj in argument
    # Need to have uploaded svg image to productimages folder
    method_obj = Method.objects.get(id=method_id)
    reaction.method_id = method_obj
    reaction.name = name
    reaction.productsmiles = product_smiles
    reaction.productimage = 'productimages/' + product_image_fn
    reaction.productname = product_name
    reaction.save()
    
    
# @shared_task
def uploadReaction(validate_output):
    # Validate output is a list - this is one way to get
    # Celery chaining to work where second function uses list output
    # from first function (validate) called
    validate_dict, validated, csv_fp, smiles_list, project_details = validate_output
    
    if not validated:
        return (validate_dict, validated)
    
    if validated:
        # Create project model
        
        for smiles in smiles_list:
            # Get retro reaction predictions from IBM API
            reactions = getRetroSyn(smiles) 
            
        
        
    
    
    def saveImage(self):
        # Function to upload product image



# def getRoboActions(reactions):
#     # Each retrosynthetic path predicted has a unique sequence_id that can
#     # be used to create a new synthesis
#     response = rxn4chemistry_wrapper.create_synthesis_from_sequence(
#         sequence_id=results['retrosynthetic_paths'][0]['sequence_id'])
#     print(response['synthesis_id'])

# # get the entire list of actions for the entire synthesis, as well as a tree representation
# synthesis_tree, ordered_tree_nodes, ordered_list_of_actions = rxn4chemistry_wrapper.get_synthesis_plan(
#     synthesis_id=response['synthesis_id']
# )
# for action in ordered_list_of_actions:
#     print(action)

In [ ]:
reactions = getRetroSyn('CC(C)C[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](CC1CCNC1=O)C(O)S(=O)(=O)O')

In [ ]:
reactions['retrosynthetic_paths'][0]

In [ ]:
reactions['retrosynthetic_paths'][0]['children']

In [ ]:
# Each retrosynthetic path predicted has a unique sequence_id that can
# be used to create a new synthesis
response = rxn4chemistry_wrapper.create_synthesis_from_sequence(
    sequence_id=reactions['retrosynthetic_paths'][0]['sequenceId'])
print(response['synthesis_id'])

# get the entire list of actions for the entire synthesis, as well as a tree representation
synthesis_tree, ordered_tree_nodes, ordered_list_of_actions = rxn4chemistry_wrapper.get_synthesis_plan(
    synthesis_id=response['synthesis_id'])
for action in ordered_list_of_actions:
    print(action)

In [ ]:
# Need to convert names to smiles!
# Use the awesome Cactus API - https://cactus.nci.nih.gov/

from urllib.request import urlopen
from urllib.parse import quote

def CIRconvert(name):
    try:
        name_converted = quote(name)
        url= 'https://cactus.nci.nih.gov/chemical/structure/' + name_converted + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'

identifiers  = ['Z-Leu-OSu']

for ids in identifiers :
    print(ids, CIRconvert(ids))